In [1]:
import requests
import pandas as pd
import requests
import numpy as np
import concurrent.futures


In [2]:
# excel_file = r"D:\Onedrive\OneDrive - UNI CONSULTING\Attachments\Code\uni solr\uni_solr\classification\ko_prod_class_22-10_with_imglink.xlsx"  # Đường dẫn đến file Excel của bạn
excel_file =  r"new_excel_file_2.xlsx"

In [3]:
df = pd.read_excel(excel_file)


In [4]:
df.head()

,STT,doc_no,doc_date,implementing_organization,hs,product_name,product_properties,decision_reason,img_links,nation,sharepoint view links,img_count,original_link,rrdcNo,Uploaded Links
0,27296,품목분류과-96,2004-01-28,관세평가분류원,8517.50-9000,IP Broadband videophone; BVP 8770,-인터넷을 통해 음성과 영상을 송수신하는 Videophone 형태의 것으로서 기존의...,본건 물품은 VoIP(Voice over Internet Protocol)기술을 이...,NaN,Korea,NaN,0,[],72004869015,NaN
1,27297,품목분류1과-848,2016-04-26,관세평가분류원,3926.90-9000,스크류 머신 흡착식 슬리브 ; MOLD SLEEVE<br/>,ο 물품개요<br/> - 스크류 체결기에서 스크류를 흡착하기 위한 슬리브 홀더에 ...,○ 관세율표 제3926호에는 “플라스틱제의 기타 제품 및 제3901호 내지 제391...,NaN,Korea,NaN,1,['/clip/framework/filedownload/kcs4gImageDownl...,72016002463,NaN
2,27298,품목분류3과-1538,2015-06-08,관세평가분류원,8473.30-9090,GLASS WINDOW(SM-T530),○ 특정 모델의 태블릿 PC용 WINDOW GLASS로 강화처리된 유리에 모서리 라...,ㅇ 관세율표 해설서 제16부 주2호 나목에서 “기타의 부분품으로서 특정한 기계 또는...,NaN,Korea,NaN,2,['/clip/framework/filedownload/kcs4gImageDownl...,72015002744,NaN
3,27299,품목분류3과-2952,2016-07-22,관세평가분류원,8536.90-9010,FEMALE TERMINAL; 7114-4112-02,ㅇ 차량 내의 기기를 연결하는 전선 및 케이블 끝에 압착 연결되어 각종 장치 및 센...,ㅇ 관세율표 해설서 제16부 주2호 가목에 “제84류나 제85류 중 어느 특정한 호...,NaN,Korea,NaN,2,['/clip/framework/filedownload/kcs4gImageDownl...,72016004496,NaN
4,27300,품목분류3과-5137,2017-09-18,관세평가분류원,8536.90-9010,Other electrical apparatus for making connecti...,ㅇ 물품의 개요 <br/> - 커넥터 하우징에 삽입되는 터미널<br/> <br/>ㅇ...,ㅇ 관세율표 제8536호에는 “전기회로의 개폐용ㆍ보호용ㆍ접속용 기기(예: 개폐기ㆍ계...,NaN,Korea,NaN,2,['/clip/framework/filedownload/kcs4gImageDownl...,72017005977,NaN


In [5]:
df.iloc[1,12]

"['/clip/framework/filedownload/kcs4gImageDownload.do?attchFileId=PCA-20160425-00000763569htCQu']"

In [6]:
df.iloc[1,11]

1

In [7]:
import os

folder_path = 'downloaded_img'  # Đường dẫn tới folder

# Lấy danh sách tất cả file trong folder
downloaded_file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

In [ ]:
file_name = "FILEIMG_PCA004M0000004895"

In [8]:
# Hàm tải ảnh từ URL về máy cục bộ
def download_image(url):
    # Lấy tên file từ URL
    url = url.strip("'\"")  # Loại bỏ ký tự thừa ở đầu và cuối
    file_name = url.split("attchFileId=")[-1]  # Trích xuất ID từ URL
    file_name = f"downloaded_img/{file_name}.jpg"  # Thêm phần mở rộng file
    if f"{file_name}.jpg" not in downloaded_file_names:
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()  # Kiểm tra lỗi HTTP

            with open(file_name, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print("Đã tải ", file_name)
            return file_name
        except Exception as e:
            print(f"Lỗi tải ảnh: {e}")
            return None

# Hàm tải ảnh và cập nhật `Uploaded Links` cho mỗi hàng
def process_row(index, row):
    uploaded_links = []
    
    if row[11] >= 1 and (pd.isna(row[14]) or not row[14] or "lỗi" in row[14]):
        image_links = row[12]
        
        if pd.notna(image_links):  # Kiểm tra ô không rỗng
            image_links = image_links.strip('][').split(', ')
            if image_links == ['']:
                return index, ''  # Nếu không có link ảnh trong hàng đó
            else:
                for link in image_links:
                    try:
                        link = link.strip("'")
                        full_path_link = "https://unipass.customs.go.kr" + link
                        local_filename = download_image(full_path_link)  # Hàm tải ảnh về
                        onlinepath = "https://yourdomain.com/uploads/"  # Thay bằng đường dẫn chính xác
                        if local_filename:
                            uploaded_links.append(onlinepath + local_filename)
                        else:
                            uploaded_links.append("lỗi")  # Đánh dấu lỗi vào danh sách nếu gặp lỗi
                    except Exception as e:
                        print(f"Lỗi khi tải ảnh từ {link}: {e}")
                        uploaded_links.append("lỗi")  # Đánh dấu lỗi vào danh sách nếu gặp lỗi

                return index, ", ".join(uploaded_links)  # Gộp các liên kết đã tải lên thành chuỗi

    return index, row[14]  # Nếu `image_links` rỗng hoặc không tải được ảnh,trả về giá trị cũ

# Hàm để xử lý mỗi phần của DataFrame với các hàng nhất định
def process_dataframe_part(part_df):
    results = []
    for index, row in part_df.iterrows():
        result = process_row(index, row)
        results.append(result)
    return results


In [9]:
# Chia DataFrame thành 50 phần nhỏ
num_threads = 50
df_split = np.array_split(df, num_threads)

# Khởi tạo ThreadPoolExecutor với 50 thread
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Xử lý từng phần của DataFrame song song
    futures = [executor.submit(process_dataframe_part, part_df) for part_df in df_split]

    # Cập nhật lại DataFrame với kết quả đã xử lý
    for future in concurrent.futures.as_completed(futures):
        results = future.result()
        for index, uploaded_links in results:
            df.at[index, 'Uploaded Links'] = uploaded_links  # Cập nhật lại cột 'Uploaded Links'

print("Hoàn tất xử lý đa luồng.")

c:\Users\nguye\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
C:\Users\nguye\AppData\Local\Temp\ipykernel_22164\1667662488.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[11] >= 1 and (pd.isna(row[14]) or not row[14] or "lỗi" in row[14]):
C:\Users\nguye\AppData\Local\Temp\ipykernel_22164\1667662488.py:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return index, row[14]  # Nếu `image_links` rỗng hoặc không tải được ảnh,trả về giá tr

Đã tải  downloaded_img/PCA-20161229-00001192720Cjv9V.jpg
Đã tải  downloaded_img/PCA-20160707-00000876354dQ24K.jpg
Đã tải  downloaded_img/PCA-20161229-00001193945lXjaW.jpg
Đã tải  downloaded_img/PCA-20201203-000039545123Finm.jpg
Đã tải  downloaded_img/PCA-20160425-00000763569htCQu.jpg
Đã tải  downloaded_img/PCA-20180518-0000233081465hYG.jpg
Đã tải  downloaded_img/PCA-20240417-00005679181mUIt8.jpg
Đã tải  downloaded_img/PCA-20231017-00005439965XIgK8.jpg
Đã tải  downloaded_img/PCA-20161229-00001185218wDDXr.jpg
Đã tải  downloaded_img/PCA-20200429-00003589888IzcnH.jpg
Đã tải  downloaded_img/PCA-20171205-00001808026QqXv3.jpg
Đã tải  downloaded_img/PCA-20170710-00001537561TI7nL.jpg
Đã tải  downloaded_img/PCA-20200527-00003626986c8Rva.jpg
Đã tải  downloaded_img/PCA-20171122-00001764843nyznI.jpg
Đã tải  downloaded_img/FILEIMG_PCA004M0000005675.jpg
Đã tải  downloaded_img/PCA-20161110-00001076954Tb9Ok.jpg
Đã tải  downloaded_img/PCA-20161229-00001186783QXr9T.jpg
Đã tải  downloaded_img/PCA-20160714

C:\Users\nguye\AppData\Local\Temp\ipykernel_22164\3254618764.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'https://yourdomain.com/uploads/downloaded_img/FILEIMG_PCA004M0000002303.jpg' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Uploaded Links'] = uploaded_links  # Cập nhật lại cột 'Uploaded Links'
C:\Users\nguye\AppData\Local\Temp\ipykernel_22164\1667662488.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[11] >= 1 and (pd.isna(row[14]) or not row[14] or "lỗi" in row[14]):
C:\Users\nguye\AppData\Local\Temp\ipykernel_22164\1667662488.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer 

Đã tải  downloaded_img/FILEIMG_PCA004M0000006594.jpg
Đã tải  downloaded_img/PCA-20161229-00001186670HU67r.jpg
Đã tải  downloaded_img/PCA-20190902-00003135102MOA8v.jpg
Đã tải  downloaded_img/PCA-20180222-00002180282mRmLX.jpg
Đã tải  downloaded_img/PCA-20161229-00001186461hUuDp.jpg
Đã tải  downloaded_img/PCA-20161229-00001193867kmKee.jpg
Đã tải  downloaded_img/PCA-20160801-00000916656TnFZN.jpg
Đã tải  downloaded_img/PCA-20201204-00003956473lENlr.jpg
Đã tải  downloaded_img/PCA-20240910-00005865512d6tMw.jpg
Đã tải  downloaded_img/PCA-20200729-000037326888g7Qc.jpg
Đã tải  downloaded_img/PCA-20161209-00001152367y4zEJ.jpg
Đã tải  downloaded_img/PCA-20181101-00002608462oKn6c.jpg
Đã tải  downloaded_img/PCA-20180309-00002201985OH87g.jpg
Đã tải  downloaded_img/PCA-20210604-00004244147845V6.jpg
Đã tải  downloaded_img/PCA-20161229-00001187157BAKG5.jpg
Đã tải  downloaded_img/FILEIMG_PCA004M0000010014.jpg
Đã tải  downloaded_img/PCA-20160819-00000942682aRh8J.jpg
Đã tải  downloaded_img/PCA-20190405-000

In [12]:
len (df)

24946

In [13]:
# Ghi lại DataFrame vào file Excel
df.to_excel(r"D:\Onedrive\OneDrive - UNI CONSULTING\Attachments\Code\uni_craw_tvpl\uni_craw_korea\threads\new_excel_file_2.xlsx", index=False)